## Siamese Network

In [1]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Activation, BatchNormalization, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Lambda
from keras import backend as K
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import sys
sys.path.append("../input/vggface/keras-vggface/keras-vggface/")
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

Using TensorFlow backend.


In [2]:
image_gen = ImageDataGenerator(rotation_range=15,
                               width_shift_range=0.05,
                               height_shift_range=0.05,
                               shear_range=0.01,
                               zoom_range=[0.9, 1.25],
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='reflect',
                               data_format='channels_last',
                               brightness_range=[0.5, 1.5])

In [3]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def load_and_align_images(filepaths, margin, image_size = 160, aug=False):
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)
        aligned = resize(img, (image_size, image_size), mode='reflect')
        if aug:
            aligned = image_gen.random_transform(aligned)
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

In [4]:
train_file_path = "../input/recognizing-faces-in-the-wild/train_relationships.csv"
train_folders_path = "../input/recognizing-faces-in-the-wild/train/"
val_famillies = "F09"

all_images = glob(train_folders_path + "*/*/*.jpg")

train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

train_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)


def gen(list_tuples, person_to_images_map, batch_size=16, aug=False):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
#         X1 = np.array([read_img(x) for x in X1])
        X1 = prewhiten(load_and_align_images(X1, margin=10, aug=aug))
        

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
#         X2 = np.array([read_img(x) for x in X2])
        X2 = prewhiten(load_and_align_images(X2, margin=10, aug=aug))
        
#         if aug:
#             labels = [lbl for lbl in labels for _ in range(2)] # change range arg based on the scale of augmented data size
        
        yield [X1, X2], labels

In [5]:
# steps per epoch
len(train)//16, len(val)//16

(191, 18)

In [6]:
def outer_product(inputs):
    """
    inputs: list of two tensors (of equal dimensions, 
        for which you need to compute the outer product
    """
    x, y = inputs
    batchSize = K.shape(x)[0]
    outerProduct = x[:,:, np.newaxis] * y[:,np.newaxis,:]
    outerProduct = K.reshape(outerProduct, (batchSize, -1))
    # returns a flattened batch-wise set of tensors
    return outerProduct

In [7]:
def baseline_model():
    input_1 = Input(shape=(160, 160, 3))
    input_2 = Input(shape=(160, 160, 3))

    base_model = load_model("../input/facenet-pretrained/facenet_keras.h5")

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    
    bilinearProduct = Lambda(outer_product, output_shape=(128**2, ))([x1, x2])
    x = Dense(100, activation="relu")(bilinearProduct)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)
    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()
    
    return model

# model = baseline_model()
model = load_model("../input/facenet-pretrained/facenet_outerprod_v13_final.h5")

In [8]:
model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(5e-9))
# for x in model.layers:
#     print(x.name, x.trainable)

In [9]:
file_path = "facenet.h5"

checkpoint = ModelCheckpoint(file_path, monitor="val_acc", verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

In [10]:
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16, aug=True), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=100, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=191, validation_steps=18)

Epoch 1/100


/opt/conda/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


 - 229s - loss: 0.3655 - acc: 0.8341 - val_loss: 0.4292 - val_acc: 0.7778

Epoch 00001: val_acc improved from -inf to 0.77778, saving model to facenet.h5
Epoch 2/100
 - 154s - loss: 0.3675 - acc: 0.8384 - val_loss: 0.4429 - val_acc: 0.8021

Epoch 00002: val_acc improved from 0.77778 to 0.80208, saving model to facenet.h5
Epoch 3/100
 - 151s - loss: 0.3821 - acc: 0.8282 - val_loss: 0.4752 - val_acc: 0.7812

Epoch 00003: val_acc did not improve from 0.80208
Epoch 4/100
 - 166s - loss: 0.4031 - acc: 0.8223 - val_loss: 0.4389 - val_acc: 0.7847

Epoch 00004: val_acc did not improve from 0.80208
Epoch 5/100
 - 172s - loss: 0.3977 - acc: 0.8243 - val_loss: 0.5626 - val_acc: 0.7292

Epoch 00005: val_acc did not improve from 0.80208
Epoch 6/100
 - 174s - loss: 0.4045 - acc: 0.8151 - val_loss: 0.4152 - val_acc: 0.7847

Epoch 00006: val_acc did not improve from 0.80208
Epoch 7/100
 - 174s - loss: 0.3821 - acc: 0.8220 - val_loss: 0.4519 - val_acc: 0.7812

Epoch 00007: val_acc did not improve from 

In [11]:
test_path = "../input/recognizing-faces-in-the-wild/test/"


def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv('../input/recognizing-faces-in-the-wild/sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
#     X1 = [x.split("-")[0] for x in batch]
#     X1 = np.array([read_img(test_path + x) for x in X1])

#     X2 = [x.split("-")[1] for x in batch]
#     X2 = np.array([read_img(test_path + x) for x in X2])

    X1 = [test_path + x.split("-")[0] for x in batch]
    X1 = prewhiten(load_and_align_images(X1, margin=10))

    X2 = [test_path + x.split("-")[1] for x in batch]
    X2 = prewhiten(load_and_align_images(X2, margin=10))

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("submission.csv", index=False)


166it [02:08,  1.25it/s]


In [12]:
from IPython.display import HTML
import base64

def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [13]:
create_download_link(submission)

In [14]:
model.save('facenet_final.h5')